In [3]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

In [4]:
df = pd.read_csv("credit_risk_dataset.csv")
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32581 entries, 0 to 32580
Data columns (total 12 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   person_age                  32581 non-null  int64  
 1   person_income               32581 non-null  int64  
 2   person_home_ownership       32581 non-null  object 
 3   person_emp_length           31686 non-null  float64
 4   loan_intent                 32581 non-null  object 
 5   loan_grade                  32581 non-null  object 
 6   loan_amnt                   32581 non-null  int64  
 7   loan_int_rate               29465 non-null  float64
 8   loan_status                 32581 non-null  int64  
 9   loan_percent_income         32581 non-null  float64
 10  cb_person_default_on_file   32581 non-null  object 
 11  cb_person_cred_hist_length  32581 non-null  int64  
dtypes: float64(3), int64(5), object(4)
memory usage: 3.0+ MB


,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_status,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length
0,22,59000,RENT,123.0,PERSONAL,D,35000,16.02,1,0.59,Y,3
1,21,9600,OWN,5.0,EDUCATION,B,1000,11.14,0,0.10,N,2
2,25,9600,MORTGAGE,1.0,MEDICAL,C,5500,12.87,1,0.57,N,3
3,23,65500,RENT,4.0,MEDICAL,C,35000,15.23,1,0.53,N,2
4,24,54400,RENT,8.0,MEDICAL,C,35000,14.27,1,0.55,Y,4


In [5]:
print(df.isnull().sum())
# person_emp_length has 895 missing values
# loan_int_rate has 3116 missing values

#calculating % of missing values in specific columns

missing_person_emp_length = df['person_emp_length'].isnull().mean()*100
missing_loan_int_rate_1 = df['loan_int_rate'].isnull().mean()*100

print(f"Missing values in dataset for variable/column 'person_emp_length': {missing_person_emp_length:.2f}%")
print(f"Missing values in dataset for variable/column 'loan_int_rate': {missing_loan_int_rate_1:.2f}%")

person_age                       0
person_income                    0
person_home_ownership            0
person_emp_length              895
loan_intent                      0
loan_grade                       0
loan_amnt                        0
loan_int_rate                 3116
loan_status                      0
loan_percent_income              0
cb_person_default_on_file        0
cb_person_cred_hist_length       0
dtype: int64
Missing values in dataset for variable/column 'person_emp_length': 2.75%
Missing values in dataset for variable/column 'loan_int_rate': 9.56%


In [6]:
# eliminate those rows having missing values for person_emp_length because missing data < 10%
df = df.dropna(subset=['person_emp_length'])

missing_person_emp_length = df['person_emp_length'].isnull().mean()*100
print(f"Missing values in 'person_emp_length' variable after eliminating: {missing_person_emp_length:.2f}%")
missing_loan_int_rate_2 = df['loan_int_rate'].isnull().mean()*100
print(f"Missing values in 'loan_int_rate' after eliminating 'person_emp_length' missing observations: {missing_loan_int_rate_2:.2f}%")
df.info()

# imputing median value for variable 'loan_int_rate'
df['loan_int_rate'] = df['loan_int_rate'].fillna(df['loan_int_rate'].median())
print(df.isnull().sum())

Missing values in 'person_emp_length' variable after eliminating: 0.00%
Missing values in 'loan_int_rate' after eliminating 'person_emp_length' missing observations: 9.62%
<class 'pandas.core.frame.DataFrame'>
Index: 31686 entries, 0 to 32580
Data columns (total 12 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   person_age                  31686 non-null  int64  
 1   person_income               31686 non-null  int64  
 2   person_home_ownership       31686 non-null  object 
 3   person_emp_length           31686 non-null  float64
 4   loan_intent                 31686 non-null  object 
 5   loan_grade                  31686 non-null  object 
 6   loan_amnt                   31686 non-null  int64  
 7   loan_int_rate               28638 non-null  float64
 8   loan_status                 31686 non-null  int64  
 9   loan_percent_income         31686 non-null  float64
 10  cb_person_default_on_file   31686 n

In [7]:
numeric_cols = ['person_age', 'person_income', 'person_emp_length', 'loan_amnt', 'loan_int_rate', 'loan_status',
                'loan_percent_income', 'cb_person_cred_hist_length']

def cal_iqr(df, column):

    q1 = df[column].quantile(0.25)
    q3 = df[column].quantile(0.75)
    iqr = q3 - q1
    lower_bound = q1 - 1.5*iqr
    upper_bound = q3 + 1.5*iqr
    outliers = df[(df[column] < lower_bound) | (df[column] > upper_bound)][column]
    return {
        'Column': column,
        'Q1': q1,
        'Q3': q3,
        'IQR': iqr,
        'Lower Bound': lower_bound,
        'Upper Bound': upper_bound,
        'Outliers Count': len(outliers),
        'Outlier (sample) Value': outliers.head().tolist()
    }

iqr_results = [cal_iqr(df, col) for col in numeric_cols]

iqr_df = pd.DataFrame(iqr_results)
print(iqr_df[['Column', 'Q1', 'Q3', 'IQR', 'Lower Bound', 'Upper Bound', 'Outliers Count']])
print('\nSample Outlier Values:\n')

for result in iqr_results:
    print(f"{result['Column']}: {result['Outlier (sample) Value']}")

                       Column        Q1        Q3       IQR  Lower Bound  \
0                  person_age     23.00     30.00      7.00       12.500   
1               person_income  39397.00  80000.00  40603.00   -21507.500   
2           person_emp_length      2.00      7.00      5.00       -5.500   
3                   loan_amnt   5000.00  12500.00   7500.00    -6250.000   
4               loan_int_rate      8.49     13.16      4.67        1.485   
5                 loan_status      0.00      0.00      0.00        0.000   
6         loan_percent_income      0.09      0.23      0.14       -0.120   
7  cb_person_cred_hist_length      3.00      8.00      5.00       -4.500   

   Upper Bound  Outliers Count  
0       40.500            1460  
1   140904.500            1464  
2       14.500             853  
3    23750.000            1636  
4       20.165              58  
5        0.000            6826  
6        0.440             615  
7       15.500            1116  

Sample Outlier Va

In [8]:
## capping limits for some variables to leverage domain knowledge

df['person_age'] = df['person_age'].clip(upper = 80)
df['person_emp_length'] = df['person_emp_length'].clip(upper = 50)
df['cb_person_cred_hist_length'] = df['cb_person_cred_hist_length'].clip(upper = 30)
df['person_income'] = np.log1p(df['person_income'])

print("Values after capping:\n")
print(df[['person_age', 'person_income', 'person_home_ownership', 'person_emp_length', 'loan_intent', 'loan_grade', 'loan_amnt',
         'loan_int_rate', 'loan_status', 'loan_percent_income', 'cb_person_default_on_file']].max())

Values after capping:

person_age                         80
person_income                15.60727
person_home_ownership            RENT
person_emp_length                50.0
loan_intent                   VENTURE
loan_grade                          G
loan_amnt                       35000
loan_int_rate                   23.22
loan_status                         1
loan_percent_income              0.83
cb_person_default_on_file           Y
dtype: object


In [9]:
from sklearn.preprocessing import LabelEncoder

categorical_columns = ['person_home_ownership', 'loan_intent', 'loan_grade', 'cb_person_default_on_file']

for col in categorical_columns:
    le = LabelEncoder()
    df[f'{col}_encoded'] = le.fit_transform(df[col])

## Feature engineering
df['debt_to_income'] = df['loan_amnt']/df['person_income']

df.to_csv('cleaned_credit_risk_dataset.csv', index = False)

df = pd.read_csv("cleaned_credit_risk_dataset.csv")
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31686 entries, 0 to 31685
Data columns (total 17 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   person_age                         31686 non-null  int64  
 1   person_income                      31686 non-null  float64
 2   person_home_ownership              31686 non-null  object 
 3   person_emp_length                  31686 non-null  float64
 4   loan_intent                        31686 non-null  object 
 5   loan_grade                         31686 non-null  object 
 6   loan_amnt                          31686 non-null  int64  
 7   loan_int_rate                      31686 non-null  float64
 8   loan_status                        31686 non-null  int64  
 9   loan_percent_income                31686 non-null  float64
 10  cb_person_default_on_file          31686 non-null  object 
 11  cb_person_cred_hist_length         31686 non-null  int

,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_status,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length,person_home_ownership_encoded,loan_intent_encoded,loan_grade_encoded,cb_person_default_on_file_encoded,debt_to_income
0,22,10.985310,RENT,50.0,PERSONAL,D,35000,16.02,1,0.59,Y,3,3,4,3,1,3186.073133
1,21,9.169623,OWN,5.0,EDUCATION,B,1000,11.14,0,0.10,N,2,2,1,1,0,109.055743
2,25,9.169623,MORTGAGE,1.0,MEDICAL,C,5500,12.87,1,0.57,N,3,0,3,2,0,599.806587
3,23,11.089821,RENT,4.0,MEDICAL,C,35000,15.23,1,0.53,N,2,3,3,2,0,3156.047422
4,24,10.904138,RENT,8.0,MEDICAL,C,35000,14.27,1,0.55,Y,4,3,3,2,1,3209.790686


In [10]:
import sqlite3

conn = sqlite3.connect('credit_risk.db')
df.to_sql('loans', conn, if_exists = 'replace', index = False)

31686

In [11]:
query = """
SELECT loan_intent, loan_grade, AVG(loan_status) AS default_rate
FROM loans
GROUP BY(loan_grade);
"""

default_rates = pd.read_sql(query,conn)
print(default_rates)

## saving queries in sql file

import os

query = """
-- Default rate by Loan grade
SELECT loan_intent, loan_grade, AVG(loan_status) AS default_rate
FROM loans
GROUP BY(loan_grade);
"""

os.makedirs('scripts', exist_ok = True)

with open('scripts/queries.sql', 'w') as f:
    f.write(query)

print("Query save to the file")

         loan_intent loan_grade  default_rate
0            VENTURE          A      0.095555
1          EDUCATION          B      0.159238
2            MEDICAL          C      0.202974
3           PERSONAL          D      0.587739
4          EDUCATION          E      0.641807
5            MEDICAL          F      0.703390
6  DEBTCONSOLIDATION          G      0.984375
Query save to the file


In [13]:
query = """
select person_age, person_income, loan_amnt, loan_status
from loans
where loan_percent_income > 0.4;
"""

high_risk_borrowers = pd.read_sql(query, conn)
print(high_risk_borrowers)

second_query = """
select person_age, person_income, loan_amnt, loan_status
from loans
where loan_percent_income > 0.4;
"""

with open('scripts/queries.sql', 'a') as f:
    f.write(second_query)

      person_age  person_income  loan_amnt  loan_status
0             22      10.985310      35000            1
1             25       9.169623       5500            1
2             23      11.089821      35000            1
3             24      10.904138      35000            1
4             26      11.252872      35000            1
...          ...            ...        ...          ...
1057          66      10.668979      22250            1
1058          60      10.727685      20000            1
1059          54      10.571343      20000            0
1060          60      10.727685      20000            1
1061          65      11.238502      35000            1

[1062 rows x 4 columns]


In [14]:
query1 = """
-- Borrower distribution by age and income bracket
SELECT 
    CASE 
        WHEN person_age < 25 THEN '18-24'
        WHEN person_age BETWEEN 25 AND 34 THEN '25-34'
        WHEN person_age BETWEEN 35 AND 44 THEN '35-44'
        WHEN person_age BETWEEN 45 AND 54 THEN '45-54'
        ELSE '55+' 
    END AS age_group,
    CASE 
        WHEN person_income < 30000 THEN '<30K'
        WHEN person_income BETWEEN 30000 AND 60000 THEN '30K-60K'
        WHEN person_income BETWEEN 60001 AND 100000 THEN '60K-100K'
        ELSE '100K+' 
    END AS income_bracket,
    COUNT(*) AS total_borrowers
FROM loans
GROUP BY age_group, income_bracket
ORDER BY age_group, income_bracket;
"""
with open('scripts/queries.sql', 'w') as f:  
    f.write(query1 + "\n\n")

borrower_distribution = pd.read_sql(query1, conn)
print(borrower_distribution.head())

  age_group income_bracket  total_borrowers
0     18-24           <30K            11941
1     25-34           <30K            15761
2     35-44           <30K             3234
3     45-54           <30K              593
4       55+           <30K              157


In [15]:
query2 = """
-- Average loan amount by intent of loan
select loan_intent,
        round(avg(loan_amnt), 2) as avg_loan_amount,
        count(*) as total_loans
from loans
group by loan_intent
order by avg_loan_amount desc;
"""

with open('scripts/queries.sql', 'a') as f:
    f.write(query2 + "\n\n")

loan_intent_summary = pd.read_sql(query2, conn)
print(loan_intent_summary.head())

         loan_intent  avg_loan_amount  total_loans
0    HOMEIMPROVEMENT         10449.22         3510
1  DEBTCONSOLIDATION          9669.59         5064
2            VENTURE          9655.95         5556
3           PERSONAL          9647.52         5369
4          EDUCATION          9522.25         6290


In [16]:
query3 = """
-- Default rate by loan grade
select loan_grade,
        count(*) as total_loans,
        sum(loan_status) as defaults,
        round(100.0 * sum(loan_status)/count(*), 2) as default_rate

from loans
group by loan_grade
order by default_rate desc;
"""

with open('scripts/queries.sql', 'a') as f:
    f.write(query3 + "\n\n")

default_by_grade = pd.read_sql(query3, conn)
print(default_by_grade.head())

  loan_grade  total_loans  defaults  default_rate
0          G           64        63         98.44
1          F          236       166         70.34
2          E          952       611         64.18
3          D         3556      2090         58.77
4          C         6321      1283         20.30


In [17]:
query4 = """
-- High Risk Borrowers (those with loan_income > 40%)
select person_age, person_income, loan_amnt, loan_percent_income, loan_status
from loans
where loan_percent_income > 0.4;
"""

with open('scripts/queries.sql', 'a') as f:
    f.write(query4 + "\n\n")

high_risk_borrowers = pd.read_sql(query4, conn)
print(high_risk_borrowers.head())

   person_age  person_income  loan_amnt  loan_percent_income  loan_status
0          22      10.985310      35000                 0.59            1
1          25       9.169623       5500                 0.57            1
2          23      11.089821      35000                 0.53            1
3          24      10.904138      35000                 0.55            1
4          26      11.252872      35000                 0.45            1


In [18]:
query5 = """
-- Default rate by employment length

select person_emp_length, count(*) as total_loans, sum(loan_status) as defaults,
        round(100.0 * sum(loan_status)/count(*), 2) as default_rate
from loans
group by person_emp_length
order by person_emp_length;
"""

with open('scripts/queries.sql', 'a') as f:
    f.write(query5 + "\n\n")

emp_length_default = pd.read_sql(query5, conn)
print(emp_length_default.head())

   person_emp_length  total_loans  defaults  default_rate
0                0.0         4105      1147         27.94
1                1.0         2915       806         27.65
2                2.0         3849       987         25.64
3                3.0         3456       696         20.14
4                4.0         2874       610         21.22


In [19]:
### Prediction for risk modelling

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X = df[['person_age', 'person_income', 'person_emp_length', 'loan_amnt', 'loan_int_rate', 
        'loan_percent_income', 'cb_person_cred_hist_length', 'debt_to_income']]
y = df['loan_status']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [20]:
### Logistic Regression

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score, confusion_matrix

#scaler = StandardScaler()
#X_train_scaled = scaler.fit_transform(X_train)
#X_test_scaled = scaler.transform(X_test)

model = LogisticRegression(max_iter = 3000)
model.fit(X_train, y_train)
#model.fit(X_train_scaled, y_train)

y_pred = model.predict(X_test)
y_prob = model.predict_proba(X_test)[:, 1]

#y_pred = model.predict(X_test_scaled)
#y_prob = model.predict_proba(X_test_scaled)[:, 1]

print(f"Accuracy: {accuracy_score(y_test, y_pred):.2f}")
print(f"Precision: {precision_score(y_test, y_pred):.2f}")
print(f"Recall: {recall_score(y_test, y_pred):.2f}")
print(f"ROC-AUC:{roc_auc_score(y_test, y_prob):.2f}")
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

# Feature importance (coefficients are now on scaled features)
importance = pd.DataFrame({'Feature': X.columns, 'Coefficient': model.coef_[0]})
print("\nFeature Importance:")
print(importance.sort_values(by='Coefficient', ascending=False))

Accuracy: 0.84
Precision: 0.70
Recall: 0.40
ROC-AUC:0.83
Confusion Matrix:
 [[4765  226]
 [ 807  540]]

Feature Importance:
                      Feature  Coefficient
5         loan_percent_income     8.803333
4               loan_int_rate     0.308755
0                  person_age     0.009501
3                   loan_amnt     0.000429
7              debt_to_income    -0.005045
6  cb_person_cred_hist_length    -0.013781
2           person_emp_length    -0.017927
1               person_income    -1.116740


In [21]:
## Decision Tree Classifier

from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

dt_model = DecisionTreeClassifier(random_state=42, max_depth=6, min_samples_split=20)
dt_model.fit(X_train, y_train)

y_pred_dt = dt_model.predict(X_test)

print("Decision Tree Performance:")
print(confusion_matrix(y_test, y_pred_dt))
print(classification_report(y_test, y_pred_dt))
print("Accuracy:", accuracy_score(y_test, y_pred_dt))

Decision Tree Performance:
[[4723  268]
 [ 605  742]]
              precision    recall  f1-score   support

           0       0.89      0.95      0.92      4991
           1       0.73      0.55      0.63      1347

    accuracy                           0.86      6338
   macro avg       0.81      0.75      0.77      6338
weighted avg       0.85      0.86      0.85      6338

Accuracy: 0.8622593878195014


In [23]:
## Random Forest

from sklearn.ensemble import RandomForestClassifier

rf_model = RandomForestClassifier(n_estimators=200, max_depth=10, random_state=42)
rf_model.fit(X_train, y_train)

y_pred_rf = rf_model.predict(X_test)

print("Random Forest Performance:")
print(confusion_matrix(y_test, y_pred_rf))
print(classification_report(y_test, y_pred_rf))
print("Accuracy:", accuracy_score(y_test, y_pred_rf))

Random Forest Performance:
[[4758  233]
 [ 586  761]]
              precision    recall  f1-score   support

           0       0.89      0.95      0.92      4991
           1       0.77      0.56      0.65      1347

    accuracy                           0.87      6338
   macro avg       0.83      0.76      0.79      6338
weighted avg       0.86      0.87      0.86      6338

Accuracy: 0.8707794256863364


In [26]:
from xgboost import XGBClassifier

xgb_model = XGBClassifier(use_label_encoder = False, eval_metric = "logloss", random_state = 42, n_estimators = 200, max_depth = 6, learning_rate = 0.1)
xgb_model.fit(X_train, y_train)

y_pred_xgb = xgb_model.predict(X_test)

print("XGBoost Performance:")
print(confusion_matrix(y_test, y_pred_xgb))
print(classification_report(y_test, y_pred_xgb))
print("Accuracy:", accuracy_score(y_test, y_pred_xgb))

XGBoost Performance:
[[4740  251]
 [ 532  815]]
              precision    recall  f1-score   support

           0       0.90      0.95      0.92      4991
           1       0.76      0.61      0.68      1347

    accuracy                           0.88      6338
   macro avg       0.83      0.78      0.80      6338
weighted avg       0.87      0.88      0.87      6338

Accuracy: 0.8764594509308931


/opt/anaconda3/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [17:12:36] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


In [27]:
import pandas as pd
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score

results = pd.DataFrame({
    "Model": ["Logistic Regression", "Decision Tree", "Random Forest", "XGBoost"],
    "Accuracy": [
        accuracy_score(y_test, model.predict(X_test)),
        accuracy_score(y_test, y_pred_dt),
        accuracy_score(y_test, y_pred_rf),
        accuracy_score(y_test, y_pred_xgb)
    ],
    "Precision": [
        precision_score(y_test, model.predict(X_test)),
        precision_score(y_test, y_pred_dt),
        precision_score(y_test, y_pred_rf),
        precision_score(y_test, y_pred_xgb)
    ],
    "Recall": [
        recall_score(y_test, model.predict(X_test)),
        recall_score(y_test, y_pred_dt),
        recall_score(y_test, y_pred_rf),
        recall_score(y_test, y_pred_xgb)
    ],
    "F1 Score": [
        f1_score(y_test, model.predict(X_test)),
        f1_score(y_test, y_pred_dt),
        f1_score(y_test, y_pred_rf),
        f1_score(y_test, y_pred_xgb)
    ],
    "ROC-AUC": [
        roc_auc_score(y_test, model.predict_proba(X_test)[:,1]),
        roc_auc_score(y_test, dt_model.predict_proba(X_test)[:,1]),
        roc_auc_score(y_test, rf_model.predict_proba(X_test)[:,1]),
        roc_auc_score(y_test, xgb_model.predict_proba(X_test)[:,1])
    ]
})

print(results)


                 Model  Accuracy  Precision    Recall  F1 Score   ROC-AUC
0  Logistic Regression  0.837015   0.704961  0.400891  0.511122  0.825341
1        Decision Tree  0.862259   0.734653  0.550854  0.629614  0.852965
2        Random Forest  0.870779   0.765594  0.564959  0.650150  0.871083
3              XGBoost  0.876459   0.764540  0.605048  0.675508  0.889361
